## 8 Puzzle 코드 (Puzzle)
- 깊이 정보 ```self.depth```
- 경로 비용 ```self.cost()``` (여기서 cost 함수는 단순히 상태의 깊이 상태를 반환, 즉 경로 비용은 모두 동일하게 1)

In [1]:
# 클래스 설계 (상태, 전이모형, 그외 함수 포함)
# 깊이 정보와 목표 상태 추가

class Puzzle:
  def __init__(self, board, depth=0):
    self.board = board # 현재 상태
    self.depth = depth # 현재 깊이

  def get_new_board(self, i1, i2, depth):
    new_board = self.board[:]
    new_board[i1], new_board[i2] = new_board[i2], new_board[i1]
    return Puzzle(new_board, depth) # 목표 및 깊이 정보 추가 & 경로 비용 업데이트 

 
  def expand(self, depth):
    result = []
    i = self.board.index(0)	
    if not i in [0, 3, 6] :		# LEFT
      result.append(self.get_new_board(i, i-1, depth))
    if not i in [0, 1, 2] :		# UP  
      result.append(self.get_new_board(i, i-3, depth))
    if not i in [2, 5, 8]:		# RIGHT 
      result.append(self.get_new_board(i, i+1, depth))
    if not i in [6, 7, 8]:		# DOWN
      result.append(self.get_new_board(i, i+3, depth))
    return result

  def cost(self):
    return self.depth

  def __str__(self):
    return str(self.board[:3]) +"\n"+\
    str(self.board[3:6]) +"\n"+\
    str(self.board[6:]) +"\n"+\
    "------------------"

  def __eq__(self, other):
    return self.board == other.board

  def __ne__(self, other):
    return self.board != other.board
  
  def __lt__(self, other):
    return self.cost() < other.cost()

  def __gt__(self, other):
    return self.cost() > other.cost()

### 균일 비용 알고리즘 위한 퍼즐 업데이트 버전 (PuzzleC)
- 경로비용 ```self.path_cost```
- ```expand()``` 함수 내 경로 비용 업데이트(경로 비용 누적)
    - 위, 아래 이동 시 3
    - 왼쪽, 오른쪽 이동 시 1

In [2]:
class PuzzleC:
    def __init__(self, board, depth=0, cost=0):
        self.board = board  # 현재 상태
        self.depth = depth  # 현재 깊이
        self.path_cost = cost  # 이동 경로 비용

    def get_new_board(self, i1, i2, depth, cost):
        new_board = self.board[:]
        new_board[i1], new_board[i2] = new_board[i2], new_board[i1]
        return PuzzleC(new_board, depth, cost)

    def expand(self, depth):
        result = []
        i = self.board.index(0)  
        # 각 이동 방향에 따른 비용 계산
        if not i in [0, 3, 6]:  # LEFT (비용 1)
            new_cost = self.path_cost + 1
            result.append(self.get_new_board(i, i-1, depth, new_cost))
        if not i in [0, 1, 2]:  # UP (비용 3)
            new_cost = self.path_cost + 3
            result.append(self.get_new_board(i, i-3, depth, new_cost))
        if not i in [2, 5, 8]:  # RIGHT (비용 1)
            new_cost = self.path_cost + 1
            result.append(self.get_new_board(i, i+1, depth, new_cost))
        if not i in [6, 7, 8]:  # DOWN (비용 3)
            new_cost = self.path_cost + 3
            result.append(self.get_new_board(i, i+3, depth, new_cost))
        return result

    def cost(self):
        return self.path_cost

    def __str__(self):
        return str(self.board[:3]) + "\n" + \
               str(self.board[3:6]) + "\n" + \
               str(self.board[6:]) + "\n" + \
               "------------------"

    def __eq__(self, other):
        return self.board == other.board

    def __ne__(self, other):
        return self.board != other.board

    def __lt__(self, other):
        return self.cost() < other.cost()

    def __gt__(self, other):
        return self.cost() > other.cost()

In [3]:
start = [2, 8, 3, 
         1, 6, 4, 
         7, 0, 5]

goal = [8, 6, 3, 
        2, 0, 4, 
        1, 7, 5]

## Uninformed Search

### 너비우선탐색(Breath-First Search)
- 루트 노드의 모든 자식 노드들을 탐색한 후에 해가 발견되지 않으면 한 레벨 내려가서 동일한 방법으로 탐색을 계속하는 방법
- FIFO 대기열 활용 (큐)


### 의사코드
```java
Function BFS(initial_state, goal_state)
  queue ← [initial_state]
  visited ← [ ]
  while queue != [ ] do
    current_state ← queue의 첫번째 요소
    if current_state == goal_state
      return SUCCESS
    else
      current_state를 visited에 추가
      current_state의 자식 노드를 생성
      if current_state의 자식 노드가 이미 queue나 visited에 있다면
        해당 자식 노드 건너뜀
      else
        남은 자식 노드들은 queue의 마지막에 추가
  return FAIL
```

In [7]:
from collections import deque

def run_bfs(initial_state, goal_state):
    queue = deque()  # FIFO 구조의 큐 사용
    visited = []

    queue.append(initial_state)

    count = 1

    while queue:
        current_state = queue.popleft()  # FIFO 구조의 큐에서 가장 처음에 들어간 왼쪽 요소부터 꺼내기
        print(f"Count:{count}, Depth:{current_state.depth}\n{current_state}")
        count += 1

        if current_state.board == goal_state:
            return "탐색 성공"

        depth = current_state.depth + 1
        visited.append(current_state)

        for state in current_state.expand(depth):
            if (state in visited) or (state in queue):
                continue
            else:
                queue.append(state)
    # 큐가 비면 탐색 실패
    return "탐색 실패"

initial_state = PuzzleC(start)
answer = run_bfs(initial_state, goal)
print(answer)

Count:1, Depth:0
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]
------------------
Count:2, Depth:1
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]
------------------
Count:3, Depth:1
[2, 8, 3]
[1, 0, 4]
[7, 6, 5]
------------------
Count:4, Depth:1
[2, 8, 3]
[1, 6, 4]
[7, 5, 0]
------------------
Count:5, Depth:2
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]
------------------
Count:6, Depth:2
[2, 8, 3]
[0, 1, 4]
[7, 6, 5]
------------------
Count:7, Depth:2
[2, 0, 3]
[1, 8, 4]
[7, 6, 5]
------------------
Count:8, Depth:2
[2, 8, 3]
[1, 4, 0]
[7, 6, 5]
------------------
Count:9, Depth:2
[2, 8, 3]
[1, 6, 0]
[7, 5, 4]
------------------
Count:10, Depth:3
[0, 8, 3]
[2, 6, 4]
[1, 7, 5]
------------------
Count:11, Depth:3
[2, 8, 3]
[6, 0, 4]
[1, 7, 5]
------------------
Count:12, Depth:3
[0, 8, 3]
[2, 1, 4]
[7, 6, 5]
------------------
Count:13, Depth:3
[2, 8, 3]
[7, 1, 4]
[0, 6, 5]
------------------
Count:14, Depth:3
[0, 2, 3]
[1, 8, 4]
[7, 6, 5]
------------------
Count:15, Depth:3
[2, 3, 0]
[1, 8, 4]
[7, 6, 5]
------------------
Coun

### 균일 비용 탐색(Uniform-Cost Search)
- 가장 적은 비용으로 목표 상태에 도달하는 경로를 찾는 데 사용하는 방법
- 우선순위 큐(Priority Queue) 대기열 활용하며, 각 노드는 튜플 사용해 (경로비용, 상태)로 표현

### 의사코드
```java
Function UCS(initial_state, goal_state)
  p_queue ← [(cost, initial_state)]
  visited ← [ ] 
  while p_queue != [ ] do
    current_cost, current_state ← p_queue 에서 경로 비용이 가장 낮은 상태
    if current_state == goal_state
      return SUCCESS
    else
      current_state를 visited에 추가 
      current_state의 자식 노드 생성
      if current_state의 자식 노드가 p_queue 이나 visited 에 있으면
        해당 자식 노드 건너뜀
      else
        자식 노드의 경로 비용을 계산 
        자식 노드들을 p_queue에 추가
return FAIL
```

In [5]:
import heapq

def run_ucs(initial_state, goal_state):
    # 우선순위 큐(pqueue) 사용
    pqueue = []
    visited = []

    # 초기 상태를 튜플을 활용해 (경로비용, 상태) 형태로 우선순위 큐에 추가
    heapq.heappush(pqueue, (initial_state.cost(), initial_state))
    
    count = 1 

    while pqueue:
        current_cost, current_state = heapq.heappop(pqueue) # 우선순위가 가장 낮은 경로비용 값을 가진 상태를 꺼낸다 (힙에서 pop)
        print(f"Count:{count}, Depth:{current_state.depth}, Cost: {current_cost} \n{current_state}")
        count += 1

        if current_state.board == goal_state:
            return "탐색 성공"

        depth = current_state.depth + 1
        visited.append(current_state)

        for state in current_state.expand(depth):
            if (state in visited) or (state in [s[1] for s in pqueue]): # 수업 시 and라고 소개했는데, or 를 쓰는게 맞습니다. 
                continue
            else:
                heapq.heappush(pqueue, (state.cost(), state))
                
    return "탐색 실패"

initial_state = PuzzleC(start) # "PuzzleC"로 비용 차이(up, down = 3; left, right = 1)있는 문제 검증
answer = run_ucs(initial_state, goal)
print(answer)

Count:1, Depth:0, Cost: 0 
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]
------------------
Count:2, Depth:1, Cost: 1 
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]
------------------
Count:3, Depth:1, Cost: 1 
[2, 8, 3]
[1, 6, 4]
[7, 5, 0]
------------------
Count:4, Depth:1, Cost: 3 
[2, 8, 3]
[1, 0, 4]
[7, 6, 5]
------------------
Count:5, Depth:2, Cost: 4 
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]
------------------
Count:6, Depth:2, Cost: 4 
[2, 8, 3]
[0, 1, 4]
[7, 6, 5]
------------------
Count:7, Depth:2, Cost: 4 
[2, 8, 3]
[1, 4, 0]
[7, 6, 5]
------------------
Count:8, Depth:2, Cost: 4 
[2, 8, 3]
[1, 6, 0]
[7, 5, 4]
------------------
Count:9, Depth:3, Cost: 5 
[2, 8, 3]
[6, 0, 4]
[1, 7, 5]
------------------
Count:10, Depth:3, Cost: 5 
[2, 8, 3]
[1, 0, 6]
[7, 5, 4]
------------------
Count:11, Depth:2, Cost: 6 
[2, 0, 3]
[1, 8, 4]
[7, 6, 5]
------------------
Count:12, Depth:4, Cost: 6 
[2, 8, 3]
[6, 4, 0]
[1, 7, 5]
------------------
Count:13, Depth:4, Cost: 6 
[2, 8, 3]
[0, 1, 6]
[7, 5, 4]
------------------
Count:14

### 깊이우선탐색(Depth-First Search) / 깊이 제한 탐색 (Depth-Limited Search)
- 항상 탐색 트리의 전선에서 가장 깊은 수준, 즉 노드들이 더이상 자식 노드가 없는 수준까지 나아간 후, 이전 노드로 후퇴하며 탐색하는 기법
- (본 실습에서는 탐색 효율을 고려하여 깊이 제한 방식으로 구현)
- LIFO 대기열 활용 (스택)

### 의사코드
```java
Function DLS(initial_state, goal_state)
  stack ← [initial_state]
  visited ← [ ]
  while stack != [ ] do
    current_state ← stack의 첫번째 요소
    if current_state == goal_state
      return SUCCESS
    if 현재 깊이 > 깊이 제한
      다음 반복으로 건너뛰기
    else
      current_state를 visited에 추가
      current_state의 자식 노드 생성
      if current_state의 자식 노드가 이미 stack이나 visited에 있다면
        해당 자식 노드 건너뜀
      else
        남은 자식 노드들은 stack의 처음에 추가
  return FAIL
```

In [6]:
from collections import deque

def run_dfs(initial_state, goal_state):
    stack = deque() # LIFO 구조의 스택 사용
    visited = [ ]

    stack.append(initial_state)
    
    count = 1  

    while stack:
        current_state = stack.pop() # LIFO 구조의 스택에서 가장 마지막으로 들어간 요소 꺼내기
        print(f"Count:{count}, Depth:{current_state.depth}\n{current_state}")
        count += 1

        if current_state.board == goal_state:
            return "탐색 성공"

        depth = current_state.depth+1
        visited.append(current_state)

        if depth > 5 : # 깊이 제약 통해 DLS를 구현
            continue

        for state in reversed(current_state.expand(depth)): # (1) reversed 를 쓰는 이유는, 탐색 시 원하는 순서대로 노드를 탐색하도록 만들려고 하는 것
            if (state in visited) or (state in stack):
                continue				    
            else: 
                stack.append(state) # (2) 위에서 reversed 때문에, [B, C, D, E]가 [E, D, C, B] 순으로 스택에 쌓이고, 이후 pop 하면 [B, C, D, E] 순으로 탐색 가능
    
    return "탐색 실패"

initial_state = Puzzle(start)
answer = run_dfs(initial_state, goal)
print(answer)

Count:1, Depth:0
[2, 8, 3]
[1, 6, 4]
[7, 0, 5]
------------------
Count:2, Depth:1
[2, 8, 3]
[1, 6, 4]
[0, 7, 5]
------------------
Count:3, Depth:2
[2, 8, 3]
[0, 6, 4]
[1, 7, 5]
------------------
Count:4, Depth:3
[0, 8, 3]
[2, 6, 4]
[1, 7, 5]
------------------
Count:5, Depth:4
[8, 0, 3]
[2, 6, 4]
[1, 7, 5]
------------------
Count:6, Depth:5
[8, 3, 0]
[2, 6, 4]
[1, 7, 5]
------------------
Count:7, Depth:5
[8, 6, 3]
[2, 0, 4]
[1, 7, 5]
------------------
탐색 성공


### 